<a href="https://colab.research.google.com/github/mohammadsjahanbakhsh/project2_quera/blob/main/finall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## preprocessing

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

In [ ]:


def transform(dataset,country_column) :
       
        
        cols_with_na = dataset.isna().sum()[dataset.isna().sum()>0].index.tolist()
        
        for col in cols_with_na : 
          
          dataset[col]=dataset.groupby([country_column])[col].transform(lambda x: x.fillna(x.mean()))
          
        cols_with_na = dataset.isna().sum()[dataset.isna().sum()>0].index.tolist()
        
        for col in cols_with_na : 
            dataset.loc[:,col].fillna(dataset[col].median(),inplace=True)
            
        
        return dataset



def transform2(dataset,drop_list):
  dataset=dataset[dataset.Year>=1950]
  dataset.drop(drop_list,axis=1,inplace=True)

  return dataset

Additional_column=["Variant","Notes","ISO3 Alpha-code","ISO2 Alpha-code","SDMX code**","Type","Location code","Parent code"]

In [ ]:
death=pd.read_csv("datasets\suicide-vs-violent-deaths.csv")

In [ ]:
death=transform2(death,['Population (historical estimates)', 'Continent',"Code"])
# death=transform(death,'Entity')

In [ ]:
vaccine=pd.read_csv("datasets\child-mortality-vs-share-of-children-immunized-against-diphtheria-pertussis-and-tetanus.csv")
vaccine=transform2(vaccine,['Population (historical estimates)', 'Continent','Code',"Mortality rate, under-5 (per 1,000 live births)"])
# vaccine=transform(vaccine,'Entity')

In [ ]:
gdp=pd.read_csv("datasets\life-expectancy-vs-gdp-per-capita.csv")
gdp.columns

In [ ]:
gdp=transform2(gdp,['Code', 'Life expectancy at birth (historical)','417485-annotations','Population (historical estimates)','Continent'])
# gdp=transform(gdp,'Entity')

In [ ]:
Estimates=pd.read_excel("datasets/WPP2022.xlsx",sheet_name="Estimates")


In [ ]:
from numpy import nan

columns=list(Estimates.iloc[15])
Estimates=Estimates[16:]
Estimates.columns=columns
Estimates=Estimates.set_index("Index")

# Estimates=Estimates.loc[Estimates.iloc[:,4].notna(),:]
Estimates.replace("...",nan,inplace=True)

In [ ]:
x=vaccine.merge(death,how="outer").merge(gdp,how="outer")

Estimates=Estimates.merge(x,left_on=["Region, subregion, country or area *","Year"],right_on=["Entity","Year"],how="left").drop("Entity",axis=1)
x

In [ ]:
hospital_bed=pd.read_csv("datasets/hospital_bed.csv")
hospital_bed.drop(["INDICATOR","SUBJECT","MEASURE","FREQUENCY","Flag Codes"],axis=1,inplace=True)
hospital_bed.rename({"TIME":"Year","Value":"hospital_bed"},axis=1,inplace=True)

In [ ]:
Estimates=Estimates.merge(hospital_bed,left_on=["ISO3 Alpha-code","Year"],right_on=["LOCATION","Year"],how="left").drop("LOCATION",axis=1)

In [ ]:

Estimates.drop(Additional_column,axis=1,inplace=True)



In [ ]:
Medium_variant=pd.read_excel("datasets/WPP2022.xlsx",sheet_name="Medium variant")

In [ ]:
columns=list(Medium_variant.iloc[15])
Medium_variant=Medium_variant[16:]
Medium_variant.columns=columns
Medium_variant=Medium_variant.set_index("Index")
Medium_variant=Medium_variant[Medium_variant.iloc[:,4].notna()]
# Medium_variant.drop(Additional_column,axis=1,inplace=True)
Medium_variant=Medium_variant[Medium_variant.Year==2022]
Medium_variant.replace("...",nan,inplace=True)

In [ ]:
Estimates.to_csv("Estimates3.csv",index=None)
# Medium_variant.to_csv("/content/drive/MyDrive/data/Medium_variant.csv",index=None)
# فایلو بدم بچه ها اشتباه داشت

In [ ]:
Estimates

In [ ]:
Estimates=pd.read_csv("Estimates.csv")
Medium_variant=pd.read_csv("Medium_variant.csv")

## model 

In [ ]:
import pandas as pd
Estimates=pd.read_csv("/content/drive/MyDrive/project2_quera/Estimates.csv")



In [ ]:


def transform(dataset,country_column) :
       
        
        cols_with_na = dataset.isna().sum()[dataset.isna().sum()>0].index.tolist()
        
        for col in cols_with_na : 
          
          dataset[col]=dataset.groupby([country_column])[col].transform(lambda x: x.fillna(x.mean()))
          
        cols_with_na = dataset.isna().sum()[dataset.isna().sum()>0].index.tolist()
        
        for col in cols_with_na : 
            dataset.loc[:,col].fillna(dataset[col].median(),inplace=True)
            
        
        return dataset


Additional_column=["Variant","Notes","ISO3 Alpha-code","ISO2 Alpha-code","SDMX code**","Type","Location code","Parent code"]


In [ ]:

Estimates.drop(Additional_column,axis=1,inplace=True)
df_est=pd.get_dummies(transform(Estimates,"Region, subregion, country or area *"),columns=["Region, subregiontry or area *"])

In [ ]:

# df_est=pd.get_dummies(Estimates,columns=["Region, subregion, country or area *"])

In [ ]:
Medium_variant=pd.read_csv("/content/drive/MyDrive/project2_quera/Medium_variant.csv")
Medium_variant=Medium_variant.drop(Additional_column,axis=1)
df_med=pd.get_dummies(transform(Medium_variant,"Region, subregion, country or area *"),columns=["Region, subregion, country or area *"])

In [ ]:
# df_est.to_csv("/content/drive/MyDrive/project2_quera/Estimates_finall.csv",index=None)


In [ ]:
x_col_est=["Life Expectancy" not in c for c in df_est.columns]
y_col_est=["Life Expectancy at Birth" in c for c in df_est.columns]
x_Estimates=df_est.loc[:,x_col_est]
y_Estimates=df_est.loc[:,y_col_est]["Life Expectancy at Birth, both sexes (years)"]


In [ ]:
x_col_med=["Life Expectancy" not in c for c in df_med.columns]
y_col_med=["Life Expectancy at Birth" in c for c in df_med.columns]
x_Medium=df_med.loc[:,x_col_med]
y_Medium=df_med.loc[:,y_col_med]["Life Expectancy at Birth, both sexes (years)"]


## predict my feature with decision tree Regressor 



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import r2_score

In [ ]:
df_est.head(2)

In [ ]:
df_est_x.head(2)

#### Predicting and adding my own fits to the ""Medium""DataSet those who are forecast more than 0.8 with R2-squares and are foreseeable in my opinion

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import StandardScaler 
my_feature_total=Estimates.columns[-5:].to_list()
my_feature=my_feature_total.copy()
r2_my_feature=dict()
medium_my_feature = dict()
my_feature2=[]



kf = KFold(n_splits=5, shuffle=True)
for i in range(len(my_feature_total)):
  j=0
  my_feature2.append(my_feature.pop(-1))
  x_last=x_Estimates.drop(my_feature_total,axis=1)
  y_last=x_Estimates[[my_feature2[-1]]]

  r2_scores2 = []
  for train_index, test_index in kf.split(x_last):
      x_train, x_test = x_last.values[train_index], x_last.values[test_index]
      y_train, y_test = y_last.values.ravel()[train_index] , y_last.values.ravel()[test_index]
    # x_train,x_test , y_train,y_test = train_test_split(x_last, y_last ,test_size=.2)

      std_scaler = StandardScaler().fit(x_train)
      x_train = std_scaler.transform(x_train)



      regressor = DecisionTreeRegressor( )
      regressor.fit(x_train, y_train)
      print(j,my_feature2[-1])
      j+=1
      y_pred = regressor.predict(std_scaler.transform(x_test))
      r2_scores2.append(r2_score(y_test, y_pred))

  r2_my_feature[my_feature2[-1]]=np.mean(r2_scores2)


  if r2_my_feature[my_feature2[-1]] > 0.8:
    medium_my_feature[my_feature2[-1]] = regressor.predict(std_scaler.transform(x_Medium.values))

print("____________________")
for i,j in r2_my_feature.items():
    print(i," : ", j)

In [ ]:
my_feature_in_medium=list(medium_my_feature.keys())
my_feature_in_medium

In [ ]:

x_Medium=pd.concat([x_Medium,pd.DataFrame(medium_my_feature)],axis=1)


In [ ]:
pd.concat([x_Medium,y_Medium],axis=1).to_csv("Medium_variant_with_my_feature.csv",index=None)


# predicting Life Expectancy at Birth, both sexes 

## decesion tree in sklearn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Medium=pd.read_csv("/content/drive/MyDrive/project2_quera/Medium_variant_with_my_feature.csv")
x_columns=["Life Expectancy" not in c for c in Medium]
y_columns=["Life Expectancy at Birth" in c for c in Medium]
x_Medium=Medium.loc[:,x_columns]
y_Medium=Medium.loc[:,y_columns]

In [ ]:
Medium.columns.shape

In [ ]:
Estimates=pd.read_csv("/content/drive/MyDrive/project2_quera/Estimates_finall.csv")

x_Estimates=Estimates.loc[:,x_Medium.columns]
y_Estimates=Estimates.loc[:,y_Medium.columns]

### predict on **Medium** dataset **before** feature selection 

In [ ]:
from time import time

In [ ]:
t1=time()
std_scaler = StandardScaler().fit(x_Estimates)
x_train = std_scaler.transform(x_Estimates)

dt = DecisionTreeRegressor(random_state=313)
dt.fit(x_train, y_Estimates)


y_pred = dt.predict(std_scaler.transform(x_Medium))

t1= time() - t1
before = r2_score(y_Medium, y_pred)
print("r2_score before feature selection :",before)
print("runTime :",t1)

In [ ]:
## time run


### **feature selection** with DecisionTree on **Estimates** dataset

In [ ]:
x_train,x_test , y_train,y_test = train_test_split(x_Estimates, y_Estimates ,test_size=.2)

std_scaler = StandardScaler().fit(x_train)
x_train = std_scaler.transform(x_train)


  
dt = DecisionTreeRegressor(random_state = 313)
dt.fit(x_train, y_train)


y_pred = dt.predict(std_scaler.transform(x_test))


r2_score(y_test, y_pred)



In [ ]:

ft_imp = pd.Series(dt.feature_importances_, index=x_Medium.columns).sort_values(ascending=False)
ft_imp = ft_imp[ft_imp >= 0.00005].sort_values(ascending=False)
ft_imp

In [ ]:

col_import=ft_imp.index.tolist() 
x_train,x_test , y_train,y_test = train_test_split(x_Estimates.loc[:,col_import], y_Estimates ,test_size=.2)

std_scaler = StandardScaler().fit(x_train)
x_train = std_scaler.transform(x_train)


  
dt = DecisionTreeRegressor(random_state = 313)
dt.fit(x_train, y_train)


y_pred = dt.predict(std_scaler.transform(x_test))


r2_score(y_test, y_pred)




### result predict on **Medium** dataset **after** feature selection 

In [ ]:
t2=time()
std_scaler = StandardScaler().fit(x_Estimates.loc[:,col_import])
x_train = std_scaler.transform(x_Estimates.loc[:,col_import])

dt = DecisionTreeRegressor(random_state=313)
dt.fit(x_train, y_Estimates)


y_pred = dt.predict(std_scaler.transform(x_Medium.loc[:,col_import]))


t2=time()-t2
print("r2_score before feature selection :",before)
print("runTime before :",t1)
print("r2_score after feature selection :",r2_score(y_Medium, y_pred))
print("runTime after:",t2)

## RandomForestRegressor in sklearn

### predict on **Medium** dataset **before** feature selection 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
t3=time()

std_scaler = StandardScaler().fit(x_Estimates)
x_train = std_scaler.transform(x_Estimates)


rf = RandomForestRegressor(n_estimators = 20)

rf.fit(x_train, y_Estimates.values.ravel());

predictions = rf.predict(std_scaler.transform(x_Medium))

t3=time()-t3
before=r2_score(y_Medium, predictions)
print("r2_score before feature selection :",r2_score(y_Medium, predictions))

In [ ]:
## run time

t3

### **feature selection** with *RandomFarest* on **Estimates** dataset

In [ ]:
x_train,x_test , y_train,y_test = train_test_split(x_Estimates, y_Estimates ,test_size=.2,random_state=313)

std_scaler = StandardScaler().fit(x_train)
x_train = std_scaler.transform(x_train)


  
rf = RandomForestRegressor(n_estimators = 20)
rf.fit(x_train, y_train.values.ravel())


y_pred = rf.predict(std_scaler.transform(x_test))


r2_score(y_test, y_pred)



In [ ]:
ft_imp = pd.Series(rf.feature_importances_, index=x_Medium.columns).sort_values(ascending=False)
ft_imp = ft_imp[ft_imp >= 0.00005].sort_values(ascending=False)
ft_imp

### result predict on **Medium** dataset **after** feature selection 

In [ ]:
t4=time()
col_import=ft_imp.index.tolist()
std_scaler = StandardScaler().fit(x_Estimates.loc[:,col_import].values)
x_train = std_scaler.transform(x_Estimates.loc[:,col_import].values)

  
rf = RandomForestRegressor(n_estimators = 20)
rf.fit(x_train, y_Estimates.values.ravel())


y_pred = rf.predict(std_scaler.transform(x_Medium.loc[:,col_import].values))


t4=time() - t4

print("r2_score before feature selection :",before)
print("runTime before :",t3)
print("r2_score after feature selection :",r2_score(y_Medium, y_pred))
print("runTime after:",t4)

## Tuning a Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV

n_estimators = list(range(10, 26))

param_grid = {
    'n_estimators': n_estimators,
}
std_scaler = StandardScaler().fit(x_Estimates.loc[:,col_import].values)
x_train = std_scaler.transform(x_Estimates.loc[:,col_import].values)

rf = RandomForestRegressor()
gs = GridSearchCV(rf, param_grid, cv=5)
gs.fit(x_train, y_Estimates.values.ravel());



In [ ]:
gs.best_params_

In [ ]:
mean_scores=sorted(gs.cv_results_['mean_test_score'].tolist())
# min = round(mean_scores[-1],5)
mean_scores

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid", palette="rainbow")
plt.figure(figsize=(15,5))
scores = gs.cv_results_['mean_test_score']
plt.plot(n_estimators, scores)
plt.xlabel("n_estimators")
plt.ylabel("accuracy")
plt.xlim(15, 25)
plt.ylim(0.993, 0.994)
plt.show()


## LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression()
std_scaler = StandardScaler().fit(x_Estimates.values)
x_train = std_scaler.transform(x_Estimates.values)

reg.fit(x_train,y_Estimates.values.ravel())
reg_pred = reg.predict(std_scaler.transform(x_Medium.values))


r2_score(y_Medium, reg_pred)

In [ ]:
reg = LinearRegression()
std_scaler = StandardScaler().fit(x_Estimates.loc[:,col_import].values)
x_train = std_scaler.transform(x_Estimates.loc[:,col_import].values)

reg.fit(x_train,y_Estimates)
reg_pred = reg.predict(std_scaler.transform(x_Medium.loc[:,col_import].values))


r2_score(y_Medium, reg_pred)

## AutoML

In [ ]:
! pip install flaml

In [ ]:
from flaml import AutoML

In [ ]:
## without feature selection 

std_scaler = StandardScaler().fit(x_Estimates.values)
x_train = std_scaler.transform(x_Estimates.values)

automl = AutoML(task='regression', time_budget=20)
automl.fit(x_train, y_Estimates.values.ravel())
y_pred = automl.predict(std_scaler.transform(x_Medium.values))
r2_score(y_Medium, y_pred)

In [ ]:
## with feature selection 

std_scaler = StandardScaler().fit(x_Estimates.loc[:,col_import].values)
x_train = std_scaler.transform(x_Estimates.loc[:,col_import].values)

automl = AutoML(task='regression', time_budget=20)
automl.fit(x_train, y_Estimates.values.ravel())
y_pred = automl.predict(std_scaler.transform(x_Medium.loc[:,col_import].values))
r2_score(y_Medium, y_pred)

## xgboost

In [ ]:

import xgboost as xgb


In [ ]:
## without feature selection 
std_scaler = StandardScaler().fit(x_Estimates.values)
x_train = std_scaler.transform(x_Estimates.values)

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 4, alpha = 10, n_estimators = 1000)

xg_reg.fit(x_train, y_Estimates.values.ravel())

xgb_pred = xg_reg.predict(std_scaler.transform(x_Medium.values))
xgb_score = r2_score(y_Medium, xgb_pred)
xgb_score

In [ ]:
## with feature selection 
std_scaler = StandardScaler().fit(x_Estimates.loc[:,col_import].values)
x_train = std_scaler.transform(x_Estimates.loc[:,col_import].values)


xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 4, alpha = 10, n_estimators = 1000)

xg_reg.fit(x_train, y_Estimates.values.ravel())

xgb_pred = xg_reg.predict(std_scaler.transform(x_Medium.loc[:,col_import].values))
xgb_score = r2_score(y_Medium, xgb_pred)
xgb_score